<a href="https://colab.research.google.com/github/romellfudi/medium/blob/main/Bard_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
#@title Import a Google ID Session (Must run it)
!pip install bardapi gradio speechbrain git+https://github.com/openai/whisper.git -q
#@markdown     Warning Do not expose the __Secure-1PSID
#@markdown 1. Visit https://bard.google.com/
#@markdown 2.  F12 for console
#@markdown 3. Session: Application → Cookies → Copy the value of __Secure-1PSID cookie.
base_system = "Im going to ask questions as normal, however your response must be concisse, do not give me extra information unless I tell you. Do not try to translate."
token='' #@param  {type: "string"}

In [ ]:
import whisper
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large']

In [ ]:
import os
import re
import whisper
import random
from speechbrain.pretrained import EncoderDecoderASR
import gradio as gr
import requests
from bardapi import Bard
from IPython.display import Audio, display, Markdown

def make_connection_to_bard(token):
    session = requests.Session()
    session.headers = {
        "Host": "bard.google.com",
        "X-Same-Domain": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
        "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8",
        "Origin": "https://bard.google.com",
        "Referer": "https://bard.google.com/",
    }
    session.cookies.set("__Secure-1PSID", token)
    return Bard(token=token, session=session)

def clean_code_blocks(full_chat):
    return re.sub(r"```[\s\S]*?```", "", full_chat)

def transcribe(file_wav, transcriptor="speechbrain"):
    global asr_model
    global whiser_base_model
    global whiser_tiny_model
    global whiser_large_model
    transcription = "."
    if transcriptor == "speechbrain":
        transcription = asr_model.transcribe_file(file_wav)
    elif transcriptor == "base_whisper":
        transcription = whiser_base_model.transcribe(file_wav)["text"]
    elif transcriptor == "tiny.en_whisper":
        transcription = whiser_tiny_model.transcribe(file_wav)["text"]
    elif transcriptor == "large_whisper":
        transcription = whiser_large_model.transcribe(file_wav)["text"]
    return transcription


chat_history = []
global asr_model
global whiser_tiny_model
global whiser_base_model
global whiser_large_model
# asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-rnnlm-librispeech", savedir="pretrained_models/asr-crdnn-rnnlm-librispeech")
whiser_base_model = whisper.load_model("base")
# whiser_tiny_model = whisper.load_model("tiny.en")
# whiser_large_model = whisper.load_model("large")
bard = make_connection_to_bard(token)
bard.get_answer(base_system)

if not os.path.exists("bard.ogg"):
    with open("bard.ogg", "wb") as f:
        f.write(bytes(bard.speech("Hello, I am Google Bard AI. Please, let's chat.")['audio']))

if not os.path.exists("bard.ogg"):
    with open("empty.ogg", "wb") as f:
        f.write(bytes(bard.speech(" ")['audio']))

# Initialize Gradio interface with custom layout
with gr.Blocks() as app:
    gr.Markdown("# Welcome to the Bard Chatbot!")
    with gr.Row():
      with gr.Column():
          gr.Markdown("## Record your message and chat with Bard.")
          audio_input = gr.Audio(label="Input Audio Channel", source="microphone", type="filepath",)
          transcriptor_options = [
              # "tiny.en_whisper",
              "base_whisper",
              # "large_whisper",
              # "speechbrain",
          ]
          transcriptor = gr.Radio(transcriptor_options, value="base_whisper", label="Transcriptor")

      with gr.Column():
          gr.Markdown("## Reproduce the Bot Answer")
          response_text = gr.Markdown()
          audio_output = gr.Audio("bard.ogg", label="Audio Output", autoplay=True)
          elapsed_time = gr.Text(label="Elapsed Time")

    with gr.Column():
        chatbot = gr.Chatbot()


    @gr.on(inputs=[audio_input, transcriptor], outputs=[audio_output, elapsed_time, chatbot])
    def chat_with_bard(file_wav, transcriptor):
        import time
        start_time = time.time()
        if file_wav is None:
          # Not modify the output just skip
          return "empty.ogg", f"{0} seg.", chat_history
        user_input = transcribe(file_wav, transcriptor)
        raw = bard.get_answer(user_input)
        bot_message = random.choice(raw['choices'])['content'][0]
        if raw['images']:
            image = random.choice(raw['images'])
            image_markdown = f"![]({image})\n\n"
        else:
            image_markdown = ""
        full_response = image_markdown+bot_message
        # print(full_response)
        chat_history.append((user_input, full_response))

        with open("bard.ogg", "wb") as f:
            f.write(bytes(bard.speech(clean_code_blocks(bot_message))['audio']))
        return "bard.ogg", f"{round(time.time() - start_time)} seg.", chat_history


# Launch the Gradio interface
app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa2bf15aae599ec90a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
